In [1]:
#!pip install git+https://github.com/LLaVA-VL/LLaVA-NeXT.git

In [2]:
#!pip install einops accelerate open-clip-torch

In [3]:
#!pip install flash-attn --no-build-isolation

In [4]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path, process_images, tokenizer_image_token
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN, IGNORE_INDEX
from llava.conversation import conv_templates, SeparatorStyle

Please install pyav to use video processing functions.


In [5]:
from PIL import Image
import requests
import copy
import torch
import sys
import warnings

In [6]:
warnings.filterwarnings("ignore")
pretrained = "lmms-lab/llava-onevision-qwen2-72b-ov"
model_name = "llava_qwen"
device = "cuda"
device_map = "auto"

In [7]:
tokenizer, model, image_processor, max_length = load_pretrained_model(pretrained, None, model_name, device_map=device_map)

Loaded LLaVA model: lmms-lab/llava-onevision-qwen2-72b-ov


tokenizer_config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/576 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.51G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/31 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/248 [00:00<?, ?B/s]

Model Class: LlavaQwenForCausalLM


In [8]:
model.eval()

LlavaQwenForCausalLM(
  (model): LlavaQwenModel(
    (embed_tokens): Embedding(151647, 8192)
    (layers): ModuleList(
      (0-79): 80 x Qwen2DecoderLayer(
        (self_attn): Qwen2FlashAttention2(
          (q_proj): Linear(in_features=8192, out_features=8192, bias=True)
          (k_proj): Linear(in_features=8192, out_features=1024, bias=True)
          (v_proj): Linear(in_features=8192, out_features=1024, bias=True)
          (o_proj): Linear(in_features=8192, out_features=8192, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=8192, out_features=29568, bias=False)
          (up_proj): Linear(in_features=8192, out_features=29568, bias=False)
          (down_proj): Linear(in_features=29568, out_features=8192, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((8192,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((8192,), eps=1e-06)
      )
    )

In [9]:
urls = [
 'https://samplebook.photos/img/4786a.jpg',
 'https://samplebook.photos/img/284v.jpg',
 'https://samplebook.photos/img/3193c.jpg',
 'https://samplebook.photos/img/948d.jpg',
 'https://samplebook.photos/img/3191i.jpg',
 'https://samplebook.photos/img/4788oo.jpg',
 'https://samplebook.photos/img/2086d.jpg',
 'https://samplebook.photos/img/4781r.jpg',
 'https://samplebook.photos/img/5521n.jpg',
 'https://samplebook.photos/img/2073cc.jpg',
 'https://samplebook.photos/img/2085b.jpg',
 'https://samplebook.photos/img/4782c.jpg',
 'https://samplebook.photos/img/4783v.jpg',
 'https://samplebook.photos/img/4912gg.jpg',
 'https://samplebook.photos/img/304h.jpg',
 'https://samplebook.photos/img/2078i.jpg',
 'https://samplebook.photos/img/2077q.jpg',
 'https://samplebook.photos/img/3192h.jpg',
 'https://samplebook.photos/img/904x.jpg',
 'https://samplebook.photos/img/4792b.jpg',
 'https://samplebook.photos/img/5301p.jpg',
 'https://samplebook.photos/img/5207j.jpg',
 'https://samplebook.photos/img/1007p.jpg',
 'https://samplebook.photos/img/17l.jpg',
 'https://samplebook.photos/img/286t.jpg',
 'https://samplebook.photos/img/287g.jpg',
 'https://samplebook.photos/img/3189j.jpg',
 'https://samplebook.photos/img/4911d.jpg',
 'https://samplebook.photos/img/4785a.jpg',
 'https://samplebook.photos/img/4793a.jpg'
]

In [10]:
prompt = 'There are two objects in this image. One is a color checker, and the other is a photographic sample book, open to a page containing a photograph. Give a detailed description of the photograph.'

In [13]:
def get_response(url):
    image = Image.open(requests.get(url, stream=True).raw)
    image_tensor = process_images([image], image_processor, model.config)
    image_tensor = [_image.to(dtype=torch.float16, device=device) for _image in image_tensor]
    
    conv_template = "qwen_1_5"  # Make sure you use correct chat template for different models
    question = DEFAULT_IMAGE_TOKEN + f'\n{prompt}'
    conv = copy.deepcopy(conv_templates[conv_template])
    conv.append_message(conv.roles[0], question)
    conv.append_message(conv.roles[1], None)
    prompt_question = conv.get_prompt()
    
    input_ids = tokenizer_image_token(prompt_question, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(device)
    image_sizes = [image.size]
    
    cont = model.generate(
        input_ids,
        images=image_tensor,
        image_sizes=image_sizes,
        do_sample=False,
        temperature=0,
        max_new_tokens=1024,
    )
    text_outputs = tokenizer.batch_decode(cont, skip_special_tokens=True)

    return text_outputs

In [14]:
responses_llava = {}

for url in urls:
    response = get_response(url)
    print(url,"\n",response,"\n\n")
    responses_llava[url] = {'lmms-lab/llava-onevision-qwen2-72b-ov':response}

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


https://samplebook.photos/img/4786a.jpg 
 ['The photograph in the sample book depicts a rugged landscape, possibly a rocky coastline or mountainous terrain. The image is in black and white, emphasizing the textures and contrasts of the natural environment. There are no visible human figures or man-made structures, suggesting a focus on the raw beauty of nature. The photograph appears to be taken during the day, as indicated by the bright lighting and clear visibility of details.'] 


https://samplebook.photos/img/284v.jpg 
 ['The photograph in the sample book depicts a large industrial structure, possibly an electrical substation or power plant. The image is in black and white, emphasizing the contrast between the metallic components and the surrounding environment. The structure features multiple cylindrical tanks, tall metal towers with intricate latticework, and numerous cables and wires connecting various parts of the facility. The ground appears to be covered with gravel or small 

In [15]:
import pickle

In [16]:
with open("responses_llava.pkl", "wb") as f:
    pickle.dump(responses_llava, f)